In [18]:
from qiskit_nature.units import *   #ANGSTROM
from qiskit_nature.second_q.drivers import *    #PySCFDriver
from qiskit_nature.second_q.mappers import * #JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.problems import EigenstateResult
from qiskit_nature.second_q.circuit.library import *  #Ansatz, HF
#from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options
from qiskit_algorithms.optimizers import * 
from qiskit_algorithms import VQE
from braket.tracking import Tracker
from qiskit_braket_provider import *
from qiskit_aer import StatevectorSimulator
from qiskit.primitives import Estimator
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_algorithms import AdaptVQE

In [12]:
driver = PySCFDriver(atom="O 0 0 -1.16; C 0 0 0; O 0 0 1.16", basis="sto-6g")
estrutura_inicial = driver.run() #Criamos a estrutura do nosso sistema

espaco_target_mid = ActiveSpaceTransformer(8,8) #Active Space para o H2
espaco_target_minimal = ActiveSpaceTransformer(4,4) #Active Space para o H2

estrutura_mid = espaco_target_mid.transform(estrutura_inicial) #mudando nossa hamiltoniana para o active space
estrutura_minimal = espaco_target_minimal.transform(estrutura_inicial)


In [13]:
op_i = estrutura_inicial.hamiltonian.second_q_op()

op_mid = estrutura_mid.hamiltonian.second_q_op()
op_minimal = estrutura_minimal.hamiltonian.second_q_op()

print('Número de operadores Fermionicos (Estrutura Inicial): ', len(op_i))

print('Número de operadores Fermionicos (Estrutura Ativa Média): ', len(op_mid))
print('Número de operadores Fermionicos (Estrutura Ativa Minima): ', len(op_minimal))

Número de operadores Fermionicos (Estrutura Inicial):  44898
Número de operadores Fermionicos (Estrutura Ativa Média):  3620
Número de operadores Fermionicos (Estrutura Ativa Minima):  456


In [14]:
#Optendo as simetrias que possuem no sistema do CO2, para criar o mapa:
mapa_default = estrutura_inicial.get_tapered_mapper(BravyiKitaevMapper())
mapa_mid = estrutura_mid.get_tapered_mapper(BravyiKitaevMapper())
mapa_minimal = estrutura_minimal.get_tapered_mapper(BravyiKitaevMapper())

obs_tapered_default = mapa_default.map(op_i)
obs_tapered_mid = mapa_mid.map(op_mid)
obs_tapered_minimal = mapa_minimal.map(op_minimal)

In [15]:
print('Profundidade de circuito ', len(obs_tapered_default))
print()
print('Profundidade de circuito ', len(obs_tapered_mid))
print()
print('Profundidade de circuito ', len(obs_tapered_minimal))

#note que são 12 qubits, ou seja diminuimos o requisito computacional.

Profundidade de circuito  16170

Profundidade de circuito  1289

Profundidade de circuito  116


In [31]:
ansatz = UCCSD(
    estrutura_mid.num_spatial_orbitals,
    estrutura_mid.num_particles,
    mapa_mid,
    initial_state=HartreeFock(
        estrutura_mid.num_spatial_orbitals,
        estrutura_mid.num_particles,
        mapa_mid))


vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters


calculo = GroundStateEigensolver(mapa_mid, vqe_solver)
resultados = calculo.solve(estrutura_mid)

In [32]:
print(resultados)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -245.354229032125
  - computed part:      -17.142188839985
  - ActiveSpaceTransformer extracted energy part: -228.21204019214
~ Nuclear repulsion energy (Hartree): 58.391968101517
> Total ground state energy (Hartree): -186.962260930608
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 8.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [None  None  None]
    - computed part:      [None  None  None]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.000000000002]
  > Dipole moment (a.u.): [None  None  None]  Total: None
                 (debye): [None  None  None]  Total: None
 


In [29]:
ansatz = UCCSD(
    estrutura_minimal.num_spatial_orbitals,
    estrutura_minimal.num_particles,
    mapa_minimal,
    initial_state=HartreeFock(
        estrutura_minimal.num_spatial_orbitals,
        estrutura_minimal.num_particles,
        mapa_minimal))



adapt_vqe = AdaptVQE(VQE(Estimator(), ansatz, SLSQP()))
adapt_vqe.supports_aux_operators = lambda: True

#Calculando o GS da molécula
calculo = GroundStateEigensolver(mapa_minimal, adapt_vqe)
resultados = calculo.solve(estrutura_minimal)

energia = resultados.groundenergy + resultados.nuclear_repulsion_energy

In [30]:
print(resultados)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -245.316024470318
  - computed part:      -4.759068543976
  - ActiveSpaceTransformer extracted energy part: -240.556955926342
~ Nuclear repulsion energy (Hartree): 58.391968101517
> Total ground state energy (Hartree): -186.9240563688
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  None]
    - computed part:      [0.0  0.0  None]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.000000000001]
  > Dipole moment (a.u.): [0.0  0.0  None]  Total: None
                 (debye): [0.0  0.0  None]  Total: None
 


In [33]:
ansatz = UCCSD(
    estrutura_minimal.num_spatial_orbitals,
    estrutura_minimal.num_particles,
    mapa_minimal,
    initial_state=HartreeFock(
        estrutura_minimal.num_spatial_orbitals,
        estrutura_minimal.num_particles,
        mapa_minimal))


vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters


calculo = GroundStateEigensolver(mapa_minimal, vqe_solver)
resultados = calculo.solve(estrutura_minimal)

In [34]:
print(resultados)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -245.316072288105
  - computed part:      -4.759116361763
  - ActiveSpaceTransformer extracted energy part: -240.556955926342
~ Nuclear repulsion energy (Hartree): 58.391968101517
> Total ground state energy (Hartree): -186.924104186588
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  None]
    - computed part:      [0.0  0.0  None]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.000000000001]
  > Dipole moment (a.u.): [0.0  0.0  None]  Total: None
                 (debye): [0.0  0.0  None]  Total: None
 
